# database upload

In [6]:
# ! pip3 install psycopg2

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import psycopg2
import datetime as dt
from delta import *
import boto3
import pprint
import yaml
import time
import json
import sys
import os

pp = pprint.PrettyPrinter(indent = 1)
print("imported modules.")

imported modules.


In [13]:
secretmanager_client = boto3.client("secretsmanager", 
                                    region_name = "us-east-2", 
                                    aws_access_key_id = os.environ["AWS_ACCESS_KEY_ID"], 
                                    aws_secret_access_key = os.environ["AWS_SECRET_ACCESS_KEY"])

response = secretmanager_client.get_secret_value(SecretId="dev/reddit/postgres")

print(response)

ClientError: An error occurred (UnrecognizedClientException) when calling the GetSecretValue operation: The security token included in the request is invalid.

## creds

In [2]:
creds_path = os.path.join("/opt", "workspace", "redditStreaming", "creds.json")

with open(creds_path, "r") as f:
    creds = json.load(f)
    print("read creds.json.")
    f.close()

read creds.json.


In [7]:
# pp.pprint(creds)

## spark

In [3]:
spark_host = "spark-master"
# spark_host = "spark-master"
aws_client = creds["aws_client"]
aws_secret = creds["aws_secret"]
index = 0
subreddit = "technology"

# initialize spark session
try:
    spark = SparkSession.builder.appName("reddit_{}".format(subreddit)) \
                .master("spark://{}:7077".format(spark_host)) \
                .config("spark.scheduler.mode", "FAIR") \
                .config("spark.scheduler.allocation.file", "file:///opt/workspace/redditStreaming/fairscheduler.xml") \
                .config("spark.executor.memory", "4096m") \
                .config("spark.executor.cores", "4") \
                .config("spark.streaming.concurrentJobs", "4") \
                .config("spark.local.dir", "/opt/workspace/tmp/driver/{}/".format(subreddit)) \
                .config("spark.worker.dir", "/opt/workspace/tmp/executor/{}/".format(subreddit)) \
                .config("spark.eventLog.enabled", "true") \
                .config("spark.eventLog.dir", "file:///opt/workspace/events/{}/".format(subreddit)) \
                .config("spark.sql.debug.maxToStringFields", 1000) \
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.hadoop:hadoop-common:3.3.1,org.apache.hadoop:hadoop-aws:3.3.1,org.apache.hadoop:hadoop-client:3.3.1,io.delta:delta-core_2.12:1.2.1,org.postgresql:postgresql:42.5.0") \
                .config("spark.hadoop.fs.s3a.access.key", aws_client) \
                .config("spark.hadoop.fs.s3a.secret.key", aws_secret) \
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
                .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider') \
                .config('spark.hadoop.fs.s3a.buffer.dir', '/opt/workspace/tmp/blocks') \
                .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
                .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
                .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
                .enableHiveSupport() \
                .getOrCreate()

    sc = spark.sparkContext
    # .config('spark.hadoop.fs.s3a.fast.upload.buffer', 'bytebuffer') \

    sc.setLogLevel('WARN')
    sc.setLocalProperty("spark.scheduler.pool", "pool{}".format(str(index)))
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsAccessKeyId", aws_client)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.awsSecretAccessKey", aws_secret)
    # sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")
    print("created spark successfully")

except Exception as e:
    print(e)

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.hadoop#hadoop-common added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.hadoop#hadoop-client added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-93c88e3c-c16f-4bbb-8686-5206a538752a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in centr

created spark successfully


## read clean df

In [33]:
def write_spark_jdbc(subreddit):
    
    df = spark.read.format("delta").option("header", True).load("s3a://reddit-streaming-stevenhurwitt/" + subreddit + "_clean")

    with open("config.yaml", "r") as g:
        config = yaml.safe_load(g)
        g.close()

    connect_str = "jdbc:postgresql://{}:5432/postgres".format(config["postgres_host"])

    try:
        df.write.format("jdbc") \
            .mode("overwrite") \
            .option("url", connect_str) \
            .option("dbtable", "public.{}".format(subreddit)) \
            .option("user", config["postgres_user"]) \
            .option("password", config["postgres_password"]) \
            .option("driver", "org.postgresql.Driver") \
            .save()

        print("wrote df to postgresql table.")

    except Exception as e:
        print(e)

In [40]:
subreddit = "BikiniBottomTwitter"
write_spark_jdbc(subreddit)

wrote df to postgresql table.


In [4]:
try:
    df = spark.read.format("delta").option("header", True).load("s3a://reddit-streaming-stevenhurwitt/" + subreddit + "_clean")

except KeyboardInterrupt:
    print("can't run at same time as streaming job...")

22/12/09 04:35:15 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
pandas_df = df.toPandas()
pandas_df.head()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,date,year,month,day
0,NaT,technology,,t2_dz54148i,False,None,0,False,How to combat the unethical and costly use of ...,r/technology,...,https://theconversation.com/how-to-combat-the-...,12257109,2022-06-23 12:19:43.780864,0,None,False,2022-06-23,2022,6,23
1,NaT,technology,"Hello Reddit! In 2015, I left the advertising ...",t2_s6k8l,False,None,0,False,"I'm Adam Roe, Founder, CEO, and Product Archit...",r/technology,...,https://www.reddit.com/r/technology/comments/v...,12258760,2022-06-23 17:01:35.214592,0,None,False,2022-06-23,2022,6,23
2,NaT,technology,,t2_8xdyr63g,False,None,0,False,Instagram is testing an AI tool that verifies ...,r/technology,...,https://www.theverge.com/2022/6/23/23179752/in...,12257384,2022-06-23 13:20:07.659520,0,None,False,2022-06-23,2022,6,23
3,NaT,technology,,t2_95cwi5om,False,None,0,False,MIT robotics engineers are accelerating robot ...,r/technology,...,https://www.pcgamer.com/mit-robotics-engineers...,12260832,2022-06-23 23:53:11.276544,1,None,False,2022-06-23,2022,6,23
4,NaT,technology,,t2_e8nv4,False,None,0,False,Netflix Layoffs Continue as 300 More Employees...,r/technology,...,https://variety.com/2022/tv/news/netflix-layof...,12258902,2022-06-23 17:48:33.786880,0,None,False,2022-06-23,2022,6,23


## write to postgres table

In [6]:
with open("config.yaml", "r") as g:
    config = yaml.safe_load(g)
    g.close()

### create schema

In [17]:
conn = psycopg2.connect(host = config["postgres_host"], user = config["postgres_user"], password = config["postgres_password"], database="postgres")


In [25]:
connect_str = "jdbc:postgresql://{}:5432/postgres".format(config["postgres_host"])

try:
    df.write.format("jdbc") \
        .mode("overwrite") \
        .option("url", connect_str) \
        .option("dbtable", "public.{}".format(subreddit)) \
        .option("user", config["postgres_user"]) \
        .option("password", config["postgres_password"]) \
        .option("driver", "org.postgresql.Driver") \
        .save()

    print("wrote df to postgresql table.")

except Exception as e:
    print(e)

wrote df to postgresql table.


## stop spark

In [24]:
try:
    spark.stop()

except Exception as e:
    print(e)

SparkSession does not exist in the JVM
